In [1]:
import numpy as np
import pandas as pd
from matplotlib import rcParams  
import matplotlib.pyplot as plt
import csv
import fiona 

%matplotlib inline
rcParams['figure.figsize'] = (10, 10) #Size of figure  
rcParams['figure.dpi'] = 125

In [2]:
Ref_year = 2013

In [5]:
pollutant_sheet_names = ('NOX Annual Area','PEC Annual Area','SO2 Annual Area','NH3 Annual Area')
Pollutant_cost = {}
for pollutant_name in pollutant_sheet_names:
    Pollutant_cost[pollutant_name] = pd.read_excel('EASIUR-POP2005-INC2005-USD2010-V20150821-XLSX/Marginal-Social-Cost.xlsx', sheetname=pollutant_name,index_col=0)

In [6]:
Pollutant_cost['NH3 Annual Area']

,1,2,3,4,5,6,7,8,9,10,...,139,140,141,142,143,144,145,146,147,148
"Column: x, Row: y",,,,,,,,,,,,,,,,,,,,,
1,513.925836,513.925836,513.925836,513.925836,513.925836,513.925836,513.925836,513.925836,513.925836,513.925836,...,513.925836,513.925836,513.925836,513.925836,513.925836,513.925836,513.925836,513.925836,513.925836,513.925836
2,513.925836,695.379678,713.861168,736.504333,791.692605,831.130187,853.422623,881.469782,894.468383,939.170927,...,1219.656416,1228.473170,1127.346834,988.810148,911.083852,849.821104,790.576650,732.726704,705.783907,513.925836
3,513.925836,744.484772,751.669004,787.677997,845.252636,883.440583,946.044506,990.701599,1024.111922,1063.582108,...,1430.250247,1346.468126,1330.337103,1192.694227,982.244904,900.104333,839.110113,759.458077,728.104641,513.925836
4,513.925836,797.817859,826.714496,888.577396,938.321937,988.925917,1058.178575,1088.935190,1116.888855,1166.689780,...,1771.060518,1637.137349,1436.759647,1564.137117,1143.789241,1130.015081,909.853415,818.820458,785.337280,513.925836
5,513.925836,909.132866,955.733638,994.891327,1020.981045,1068.614515,1128.749324,1170.652975,1176.664007,1214.165770,...,1790.334174,1781.488267,1671.361536,1735.710100,1591.801119,1343.886652,1052.970212,915.768108,843.714620,513.925836
6,513.925836,1018.018486,1035.149515,1055.395935,1075.992795,1120.348463,1165.903860,1221.604589,1247.984108,1285.442741,...,1932.602411,1959.923226,1340.675741,1465.228904,1657.070774,1132.698659,1111.847090,1068.608290,967.688122,513.925836
7,513.925836,1073.919466,1098.822887,1105.501453,1122.539658,1182.090038,1232.730508,1282.441405,1326.859520,1362.697600,...,2110.829414,1926.048170,1803.045352,1567.994286,1436.463236,1406.093094,1380.094970,1168.559820,1089.168793,513.925836
8,513.925836,1108.440975,1148.150325,1162.586429,1186.971069,1232.962119,1292.697536,1352.888457,1384.696944,1394.983201,...,2602.389403,2327.467420,2023.902692,1839.152830,1716.012497,1579.545492,1504.129081,1367.945179,1257.556469,513.925836
9,513.925836,1178.168180,1205.213659,1227.705706,1260.168863,1303.358968,1362.544879,1399.665588,1416.358050,1429.322851,...,2707.918512,2439.382209,2235.362559,2066.664191,2011.695615,1826.349089,1686.912754,1356.223619,1480.196000,513.925836


In [8]:
# Read fleet age
CAFE_fleet_age = pd.read_excel('parameters_2016-05-12.xlsx', sheetname='Historic Fleet Data')
CAFE_fleet_age.columns = CAFE_fleet_age.loc[1]
Vehicles_by_year = {}

for model_year in range(1975,Ref_year+1):
    veh_age = Ref_year-model_year+1
    _entry = CAFE_fleet_age.loc[np.logical_and(CAFE_fleet_age['Model Year']==model_year,CAFE_fleet_age['Vehicle Age']==veh_age)]
    assert len(_entry['LDV'])==1
    Vehicles_by_year[model_year] = _entry['LDV'].values[0]
#plt.plot(Vehicles_by_year.keys(),Vehicles_by_year.values())

In [9]:
# Read fuel economy data
CAFE_fuel_economy = pd.read_excel('parameters_2016-05-12.xlsx', sheetname='Fuel Economy Data',header=[2,3])
Fuel_economy_by_year = CAFE_fuel_economy['LDV (Passenger Cars)'][u'Gasoline\nFE']

In [10]:
# Average fuel economy
num_vehicles = 0.
fleet_fuel_economy_mpg = 0.
for model_year in range(1975,Ref_year+1):
    num_vehicles+=Vehicles_by_year[model_year]
    fleet_fuel_economy_mpg+=Fuel_economy_by_year[model_year]*Vehicles_by_year[model_year]
fleet_fuel_economy_mpg = fleet_fuel_economy_mpg/float(num_vehicles)
print fleet_fuel_economy_mpg

31.22396717321215


In [11]:
# Gasoline properties
gal_per_liter = 0.264172
kg_per_liter = 0.755

Car_emissions = pd.read_excel('GasolineEmissionData.xlsx')
# These are in grams per kg of gas
Car_emissions = Car_emissions.loc[Car_emissions['Category']=='gasoline, fossile']

# Emissions are in kg/mile. Computed as g/kg (table output) * kg/g * kg/l * l/gal * gal/mile 
CO2_emissions_per_mile = Car_emissions['CO2/carbon dioxide'].values[0]/1000.*kg_per_liter/gal_per_liter/fleet_fuel_economy_mpg
CO_emissions_per_mile = Car_emissions['CO/carbon monoxide'].values[0]/1000.*kg_per_liter/gal_per_liter/fleet_fuel_economy_mpg
CH4_emissions_per_mile = Car_emissions['CH4/methane'].values[0]/1000.*kg_per_liter/gal_per_liter/fleet_fuel_economy_mpg
NOx_emissions_per_mile = Car_emissions['NOx/nitrogen oxides - NOx as NO2'].values[0]/1000.*kg_per_liter/gal_per_liter/fleet_fuel_economy_mpg
PM_emissions_per_mile = Car_emissions['Particulates'].values[0]/1000.*kg_per_liter/gal_per_liter/fleet_fuel_economy_mpg
SO2_emissions_per_mile = Car_emissions['SO2/sulfur dioxide'].values[0]/1000.*kg_per_liter/gal_per_liter/fleet_fuel_economy_mpg
VOC_emissions_per_mile  = Car_emissions['VOCs (NMVOC+VOC)'].values[0]/1000.*kg_per_liter/gal_per_liter/fleet_fuel_economy_mpg

NH3_emissions_per_mile = 0 #No data


In [12]:
CO2_cost = 105 #https://19january2017snapshot.epa.gov/climatechange/social-cost-carbon_.html
CO_cost = 0. #COME BACK TO THIS
CH4_cost = 2800. #same as above

#Costs are in $/ton. Hence the /1000 below

Social_cost_per_mile = dict()

for row in Pollutant_cost['NH3 Annual Area'].columns:
    for col in Pollutant_cost['NH3 Annual Area'].index:
        _key = str(col)+'_'+str(row)
        
        Social_cost_per_mile[_key] = Pollutant_cost['NOX Annual Area'][row].loc[col]*NOx_emissions_per_mile/1000.  #$/ton * kg/mile * ton/kg
        Social_cost_per_mile[_key] += Pollutant_cost['PEC Annual Area'][row].loc[col]*PM_emissions_per_mile/1000.
        Social_cost_per_mile[_key] += Pollutant_cost['SO2 Annual Area'][row].loc[col]*SO2_emissions_per_mile/1000.
        Social_cost_per_mile[_key] += Pollutant_cost['NH3 Annual Area'][row].loc[col]*NH3_emissions_per_mile/1000.
        Social_cost_per_mile[_key] += CO2_cost*CO2_emissions_per_mile/1000.
        Social_cost_per_mile[_key] += CO_cost*CO_emissions_per_mile/1000.
        Social_cost_per_mile[_key] += CH4_cost*CH4_emissions_per_mile/1000.
        


In [13]:
Social_cost_per_mile

{'18_85': 0.04395848763850572,
 '62_39': 0.042306554170048964,
 '62_38': 0.04165996849882532,
 '62_37': 0.04186593745105313,
 '62_36': 0.041358490429953314,
 '62_35': 0.041296465508998535,
 '62_34': 0.04125729688876274,
 '62_33': 0.04123964773701365,
 '62_32': 0.041124724382140584,
 '62_31': 0.04114919430993876,
 '62_30': 0.04126866897032644,
 '31_82': 0.051469751631184585,
 '31_83': 0.05095649218896546,
 '31_80': 0.0520435599176096,
 '31_81': 0.05188006922296701,
 '31_86': 0.05005588304120981,
 '31_87': 0.05042834712252767,
 '31_84': 0.050792715070837824,
 '31_85': 0.05020621994819777,
 '31_88': 0.05021567029654102,
 '31_89': 0.05051873518216871,
 '82_145': 0.05207491964887477,
 '3_106': 0.04151806641111137,
 '100_136': 0.04896390706775632,
 '17_137': 0.043087321502629734,
 '91_113': 0.05051606968283905,
 '91_112': 0.050761168529154174,
 '91_111': 0.050931776493998916,
 '91_110': 0.050796286667144856,
 '91_117': 0.04958870768701293,
 '40_62': 0.04290570388825646,
 '91_115': 0.05048152

In [33]:
with open('Social_cost_of_driving_per_mile.csv','wb') as csvfile:
    csvwriter = csv.writer(csvfile,delimiter=',')
    csvwriter.writerow(['ROW_COL','usd_per_mile'])
    for key, val in Social_cost_per_mile.iteritems():
        csvwriter.writerow([key,val])

In [ ]:
DFW_tract_clusters_social_driving_cost

In [14]:
area_by_cluster = dict()
cost_by_cluster = dict()
with fiona.open('maps/DFW_tract_clusters_with_social_driving_costs_and_area_50cl.shp') as shapefile:
    for f in shapefile:
        cluster = f['properties']['DFW_tract_']
        if cluster not in area_by_cluster.keys():
            area_by_cluster[cluster] = 0
            cost_by_cluster[cluster] = 0
        area_by_cluster[cluster] += f['properties']['area']
        if f['properties']['SCDPM_DM'] is not None:
            cost_by_cluster[cluster] += f['properties']['area']*f['properties']['SCDPM_DM']

clusters = area_by_cluster.keys()
for cluster in clusters:
    cost_by_cluster[cluster] = cost_by_cluster[cluster]/area_by_cluster[cluster]

In [15]:
# This is the driving cost in USD/mile per cluster for ICE-engined car.
cost_by_cluster

{u'0': 0.06488147455867084,
 u'1': 0.052083653122285495,
 u'10': 0.06627999999999999,
 u'11': 0.05468671247357295,
 u'12': 0.06469009823182709,
 u'13': 0.06279,
 u'14': 0.058418059862187746,
 u'15': 0.06603412605588044,
 u'16': 0.06230861983214175,
 u'17': 0.05736097924773022,
 u'18': 0.05250933626549379,
 u'19': 0.06628,
 u'2': 0.06099999999999997,
 u'20': 0.061518939157566344,
 u'21': 0.06493206993006993,
 u'22': 0.06250912272404824,
 u'23': 0.05401906065589628,
 u'24': 0.0648355066921606,
 u'25': 0.06627999999999999,
 u'26': 0.0640157618836073,
 u'27': 0.055438205153519604,
 u'28': 0.06015677539608574,
 u'29': 0.06508999999999998,
 u'3': 0.05784046848928251,
 u'30': 0.05263194441782463,
 u'31': 0.057022874956094136,
 u'32': 0.05744469825362908,
 u'33': 0.06456,
 u'34': 0.05969656077348065,
 u'35': 0.06266918818272098,
 u'36': 0.056682457185405793,
 u'37': 0.05681928788097647,
 u'38': 0.053643398372389454,
 u'39': 0.06509000000000001,
 u'4': 0.06385780658436215,
 u'40': 0.06530527756

In [16]:
with open('Social_cost_of_driving_per_cluster_50cl.csv','wb') as csvfile:
    csvwriter = csv.writer(csvfile,delimiter=',')
    csvwriter.writerow(['Cluster','usd_per_mile'])
    for key, val in cost_by_cluster.iteritems():
        csvwriter.writerow([key,val])

In [17]:
cost_by_cluster

{u'0': 0.06488147455867084,
 u'1': 0.052083653122285495,
 u'10': 0.06627999999999999,
 u'11': 0.05468671247357295,
 u'12': 0.06469009823182709,
 u'13': 0.06279,
 u'14': 0.058418059862187746,
 u'15': 0.06603412605588044,
 u'16': 0.06230861983214175,
 u'17': 0.05736097924773022,
 u'18': 0.05250933626549379,
 u'19': 0.06628,
 u'2': 0.06099999999999997,
 u'20': 0.061518939157566344,
 u'21': 0.06493206993006993,
 u'22': 0.06250912272404824,
 u'23': 0.05401906065589628,
 u'24': 0.0648355066921606,
 u'25': 0.06627999999999999,
 u'26': 0.0640157618836073,
 u'27': 0.055438205153519604,
 u'28': 0.06015677539608574,
 u'29': 0.06508999999999998,
 u'3': 0.05784046848928251,
 u'30': 0.05263194441782463,
 u'31': 0.057022874956094136,
 u'32': 0.05744469825362908,
 u'33': 0.06456,
 u'34': 0.05969656077348065,
 u'35': 0.06266918818272098,
 u'36': 0.056682457185405793,
 u'37': 0.05681928788097647,
 u'38': 0.053643398372389454,
 u'39': 0.06509000000000001,
 u'4': 0.06385780658436215,
 u'40': 0.06530527756